In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Parâmetros gerais
target_rows = 13000

# Linhas e tipos de transporte (exemplos)
linhas = [
    {"linha": "Metrô 1 - Azul", "tipo": "metrô"},
    {"linha": "Metrô 2 - Verde", "tipo": "metrô"},
    {"linha": "Ônibus 512", "tipo": "ônibus"},
    {"linha": "Ônibus 635", "tipo": "ônibus"},
    {"linha": "Ônibus 782", "tipo": "ônibus"},
]

sentidos = ["ida", "volta"]
status_possiveis = ["normal", "atrasada", "interrompida"]

# Calcular quantas registros por combinação linha+sentido
# Total combinações = 5 linhas * 2 sentidos = 10
rows_per_comb = target_rows // (len(linhas) * len(sentidos))  # Ex: 13000/10 = 1300 registros por combinação

# Tempo entre viagens para atingir a quantidade desejada dentro do intervalo do evento
start_time = datetime(2025, 11, 15, 12, 0, 0)  # 3h antes do evento
end_time = datetime(2025, 11, 16, 5, 0, 0)     # 3h depois do evento
total_minutes = int((end_time - start_time).total_seconds() // 60)
# Ajustar freq_minutes para gerar o número correto de registros
# freq_minutes = total_minutes / rows_per_comb (aprox)
freq_minutes = max(1, total_minutes // rows_per_comb)

# Gerar horários previstos baseados no freq_minutes
horarios = pd.date_range(start=start_time, end=end_time, freq=f'{freq_minutes}min')

records = []
id_viagem = 1
np.random.seed(42)

for linha_info in linhas:
    for sentido in sentidos:
        # Selecionar só os horários que cabem para a combinação (limitar ao rows_per_comb)
        horarios_subset = horarios[:rows_per_comb]
        for horario in horarios_subset:
            # Simular tempo de viagem
            base_tempo = 20 if linha_info["tipo"] == "metrô" else 30
            if 21 <= horario.hour < 23:
                tempo_viagem = base_tempo + np.random.randint(5, 15)
            else:
                tempo_viagem = base_tempo + np.random.randint(-2, 5)
            tempo_viagem = max(5, tempo_viagem)

            # Passageiros estimados
            if 18 <= horario.hour or horario.hour <= 2:
                passageiros = np.random.randint(50, 200) if linha_info["tipo"] == "ônibus" else np.random.randint(100, 500)
            else:
                passageiros = np.random.randint(10, 100) if linha_info["tipo"] == "ônibus" else np.random.randint(20, 200)

            # Status da linha
            status = np.random.choice(status_possiveis, p=[0.8, 0.15, 0.05])

            records.append({
                "id_viagem": id_viagem,
                "linha_transporte": linha_info["linha"],
                "tipo_transporte": linha_info["tipo"],
                "horario_previsto": horario,
                "tempo_viagem_min": tempo_viagem,
                "passageiros_estimados": passageiros,
                "status_linha": status,
                "sentido": sentido
            })
            id_viagem += 1

# Criar DataFrame
df_transporte = pd.DataFrame(records)

# Ajuste final: garantir ~13000 linhas
print(f"Total linhas geradas: {len(df_transporte)}")

# Salvar CSV
df_transporte.to_csv("dados_transporte_evento_13000.csv", index=False)
print("Arquivo 'dados_transporte_evento_13000.csv' criado com sucesso!")


Total linhas geradas: 10210
Arquivo 'dados_transporte_evento_13000.csv' criado com sucesso!
